In [1]:
import numpy as np
import plotly.express as px
import pandas as pd 
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import math 
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import datasets
from sklearn.metrics import zero_one_loss
from sklearn.model_selection import KFold

In [83]:
class SelftrainKfold:
    def __init__(self):
        self.error = []
        pass
    def readfile(self, path):
        self.raw_data = pd.read_csv(path, header=None)
        self.raw_data = self.raw_data.sample(frac=1).reset_index(drop=True) # shuffle the dataframe in-place and reset the index
        print(self.raw_data)
    def splitdata(self):
        self.X = self.raw_data.iloc[:,:-1] #get all columns except the last one
        self.y = self.raw_data.iloc[:,-1]
        self.X = self.X.to_numpy()
        self.y = self.y.to_numpy()
        
    def fitwithKfold(self):
        self.kfold = KFold(n_splits= 5, shuffle= True, random_state = 1)
        i=1
        for train_index, test_index in self.kfold.split(self.X):
        #     print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test =  self.X[train_index],  self.X[test_index]
            y_train, y_test =  self.y[train_index],  self.y[test_index]

            ## 15% of the training data is labeled
            X_train_lab, X_unl, y_train_lab, _ = train_test_split(X_train, y_train, train_size=0.15, random_state=1)
            # 1. Training on the labeled dataset
            print("Fold_{}".format(i))
            while True:
                #Step1
                clf = svm.SVC(kernel='linear', probability=True,C=1).fit(X_train_lab, y_train_lab)
                print ('Accuracy: ',clf.score(X_test, y_test), ' error: ', 1 - clf.score(X_test, y_test) )
                if (len(X_unl) == 0): # exit the training if no more unlabeled data
        #             print("fist break")
                    break
                #Step2
                clp= clf.predict_proba(X_unl)
                clf_prob = pd.DataFrame(clp)
                lab=clf.predict(X_unl)
                clf_prob["max"] = clf_prob.max(axis = 1)
                clf_prob["label"] = lab
            #     print (clf_prob)
                th = 0.8
                if len(X_unl[clf_prob["max"] > th]) == 0: #exit if no more samples meets the threshold condition
                    break
                #Step3
                #add the predicted labels to the training dataset
                X_train_new = np.append(X_train_lab, X_unl[clf_prob["max"] > th], axis=0)
                y_train_new = np.append(y_train_lab, clf_prob['label'][clf_prob["max"] > th].values, axis=0)
                X_train_lab = X_train_new
                y_train_lab = y_train_new

                #remove the added labels from the unlabled dataset
                X_unl_df = pd.DataFrame(X_unl)
                X_unl_df = X_unl_df.drop(X_unl_df[clf_prob["max"] > th].index).reset_index(drop=True)
                #update the unlabeled set
                X_unl = X_unl_df.values

            print("Fold_{}_done".format(i))
            self.error.append(clf.score(X_test, y_test))
            i+=1
    def ave_error(self):
        return sum(self.error)/len(self.error)
        

In [84]:
obj = SelftrainKfold()

In [85]:
obj.readfile("/home/safwan/Documents/spring2021/ece523/hw/hw5/breast-cancer.csv")

            0         1         2         3         4        5         6  \
0   -0.656576  0.912871  0.531300 -0.455814  0.508429 -1.42093 -0.937281   
1    0.331744 -0.912871  2.430700 -0.455814  0.508429 -1.42093  1.063180   
2   -1.644900  0.912871  1.481000  0.413271  0.508429  1.28831  1.063180   
3    0.331744 -0.912871  0.531300  0.413271  0.508429  1.28831 -0.937281   
4    0.331744 -0.912871 -0.893249 -0.455814  0.508429 -0.06631  1.063180   
..        ...       ...       ...       ...       ...      ...       ...   
281  0.331744 -0.912871 -0.418399 -0.455814  0.508429 -1.42093  1.063180   
282 -0.656576  0.912871  1.006150 -0.455814 -1.511170  1.28831  1.063180   
283  1.320060 -0.912871 -1.368100 -0.455814  0.508429 -0.06631  1.063180   
284 -0.656576  0.912871  0.056451 -0.455814  0.508429 -0.06631 -0.937281   
285 -0.656576  0.912871  0.056451 -0.455814  0.508429 -0.06631  1.063180   

            7         8  9  
0   -0.130877  0.557527  0  
1    0.700918  0.557527  0  


In [86]:
obj.splitdata()

In [87]:
obj.fitwithKfold()

Fold_1
Accuracy:  0.7758620689655172  error:  0.22413793103448276
Accuracy:  0.7758620689655172  error:  0.22413793103448276
Accuracy:  0.7758620689655172  error:  0.22413793103448276
Accuracy:  0.7758620689655172  error:  0.22413793103448276
Accuracy:  0.7413793103448276  error:  0.2586206896551724
Accuracy:  0.7758620689655172  error:  0.22413793103448276
Accuracy:  0.7758620689655172  error:  0.22413793103448276
Accuracy:  0.7758620689655172  error:  0.22413793103448276
Accuracy:  0.7586206896551724  error:  0.24137931034482762
Accuracy:  0.7586206896551724  error:  0.24137931034482762
Fold_1_done
Fold_2
Accuracy:  0.5087719298245614  error:  0.49122807017543857
Accuracy:  0.5087719298245614  error:  0.49122807017543857
Accuracy:  0.5087719298245614  error:  0.49122807017543857
Accuracy:  0.5263157894736842  error:  0.4736842105263158
Accuracy:  0.5263157894736842  error:  0.4736842105263158
Accuracy:  0.5263157894736842  error:  0.4736842105263158
Fold_2_done
Fold_3
Accuracy:  0.80

In [88]:
obj.ave_error()

0.6885662431941924

### For the results above, we can see that the self training did help in 3 out of 5 folds and the boost in performance was not significat. However, we can also observe that in some cases the self training algorithm gave worst accuracy compared to the previous iteration.

In [90]:
# Using different dataset
obj1 = SelftrainKfold()
obj1.readfile('/home/safwan/Documents/spring2021/ece523/hw/hw5/abalone.csv')

             0         1         2         3         4         5         6  \
0     0.053792  0.882716  0.928243  0.489721  0.797852  0.590786  1.089330   
1     1.261790 -1.781890 -1.842820 -1.542460 -1.373890 -1.326200 -1.309990   
2     1.261790 -1.823520 -1.893200 -1.662000 -1.402440 -1.339720 -1.364730   
3     0.053792  0.466371  0.525180  0.250642  0.584756  0.892638  0.409677   
4    -1.154210  1.049250  1.079390  1.087420  1.112910  1.759900 -0.000854   
...        ...       ...       ...       ...       ...       ...       ...   
4172 -1.154210  1.299060  1.331310  1.206960  1.619650  1.608970  1.581970   
4173 -1.154210  1.757040  1.784750  1.087420  2.641290  2.669960  3.370060   
4174  0.053792  0.882716  0.877860  1.446040  1.216910  1.349920  1.417760   
4175  1.261790 -0.324683 -0.230565 -0.227518 -0.531705 -0.490476 -0.023661   
4176 -1.154210  1.299060  1.331310  1.326500  1.944900  0.888133  1.303720   

             7  8  
0     0.762695  2  
1    -1.453500  0  
2  

In [91]:
obj1.splitdata()
obj1.fitwithKfold()

Fold_1
Accuracy:  0.645933014354067  error:  0.354066985645933
Accuracy:  0.645933014354067  error:  0.354066985645933
Accuracy:  0.645933014354067  error:  0.354066985645933
Accuracy:  0.6423444976076556  error:  0.35765550239234445
Accuracy:  0.6495215311004785  error:  0.3504784688995215
Accuracy:  0.6435406698564593  error:  0.3564593301435407
Accuracy:  0.6399521531100478  error:  0.36004784688995217
Accuracy:  0.638755980861244  error:  0.36124401913875603
Accuracy:  0.6399521531100478  error:  0.36004784688995217
Accuracy:  0.6351674641148325  error:  0.3648325358851675
Accuracy:  0.6339712918660287  error:  0.36602870813397126
Accuracy:  0.6339712918660287  error:  0.36602870813397126
Accuracy:  0.6351674641148325  error:  0.3648325358851675
Fold_1_done
Fold_2
Accuracy:  0.6435406698564593  error:  0.3564593301435407
Accuracy:  0.6435406698564593  error:  0.3564593301435407
Accuracy:  0.6447368421052632  error:  0.35526315789473684
Accuracy:  0.6507177033492823  error:  0.34928

In [93]:
obj1.ave_error()

0.6152734721943673

### We see similar observation to the previous resutls